This notebook is dedicated to exploring chunking and breaking up the resume by section.

The header or professional summary (if included) should be evaluated on an individual basis and possibly thrown out.

In [31]:

from langchain.chains import LLMChain


In [1]:
# get local docs
with open("resume.md") as f:
    resume = f.read()
with open("job_desc.md") as f:
    job_desc = f.read()
job_title = "django engineer"


# Chunking job sections

## w/ LangChain Documents

I think keeping the sections as paragraphs would be best here

In [3]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers = [
    ('#', 'Header 1'),
    ('##', 'Header 2'),
    ('###', 'Header 3'),
    ('####', 'Header 4'),
]
md_spitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers)

In [4]:
splits = md_spitter.split_text(resume)

In [10]:
splits[-3]

Document(page_content="Responsible for interpreting blueprints and formulating bid proposals for large scale commercial projects. Additionally responsible for communicating with contractors and vendors for Requests for Proposals (RFP's) and Request for Quotations (RFQ's).  \n- Achieved competency in mechanical blueprint interpretation, bid formulation, and project lifecycle with half of the time of the typical training pipeline which provided value to the company and team members.\n- Developed and implemented software in Python to assist with routine tasks which increased productivity of a 2 man team threefold.\n- Represented the company at a networking event which opened the door to new contractors and bids for upcoming projects in the greater metropolitan area.\n- Communicated with lead contractor during an onsite walkthrough which lead to successful awarding of a Rutgers Camden Lab renovation.", metadata={'Header 1': 'Experience', 'Header 2': 'Estimating Engineer, Camden Mechanical 

Once you split the documents, how do you feed them back to the LLM?

In [20]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=.85, model_name="gpt-3.5-turbo")

In [28]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate


system_message = SystemMessagePromptTemplate.from_template(
    "You're an expert career consultant with an IQ over 140. Please improve this resume section for this {job_title} position."
    "Improve the section by matching grammatical syntax and lexicon."
    "This is the job description:\n{job_desc}")
job_desc_prompt = HumanMessagePromptTemplate.from_template("Here is the resume section:\n{section}")
chat_prompt = ChatPromptTemplate.from_messages([system_message, job_desc_prompt])

messages = chat_prompt.format_prompt(section=splits[-3],
                                     job_desc=job_desc,
                                     job_title=job_title).to_messages()


In [29]:
response = chat_model.predict_messages(messages)

In [30]:
response

AIMessage(content='Experience:\n\nEstimating Engineer, Camden Mechanical (MWBE), Jul 2014 - Nov 2016 (2 years 5 months)\n\n- Responsible for interpreting blueprints and formulating bid proposals for large-scale commercial projects.\n- Communicated with contractors and vendors for Requests for Proposals (RFPs) and Request for Quotations (RFQs).\n- Achieved competency in mechanical blueprint interpretation, bid formulation, and project lifecycle in half the time of the typical training pipeline, providing value to the company and team members.\n- Developed and implemented Python software to streamline routine tasks, resulting in a threefold increase in productivity for a 2-man team.\n- Represented the company at a networking event, facilitating connections with new contractors and bids for upcoming projects in the greater metropolitan area.\n- Collaborated with lead contractors during an onsite walkthrough, leading to the successful awarding of a Rutgers Camden Lab renovation project.', 

Turning this into a chain and ditching the use of chat templates

In [32]:
gram_prompt = PromptTemplate.from_template("""
You're an expert career consultant with an IQ over 140.
Please improve this resume section for this {job_title} position.
Improve the section by matching grammatical syntax and lexicon.

This is the job description:\n\n{job_desc}.
\n\n
Here is the resume section:\n{section}
""")

grammatical_chain = LLMChain(prompt=gram_prompt, llm=ChatOpenAI(temperature=.85, model_name="gpt-3.5-turbo"))

In [33]:
grammatical_chain.predict(section=splits[-3], job_desc=job_desc, job_title=job_title)


'Experience:\n\nEstimating Engineer, Camden Mechanical (MWBE), Jul 2014 - Nov 2016 (2 years 5 months)\n\n- Interpreted blueprints and formulated bid proposals for large-scale commercial projects.\n- Communicated with contractors and vendors for Requests for Proposals (RFPs) and Request for Quotations (RFQs).\n- Achieved competency in mechanical blueprint interpretation, bid formulation, and project lifecycle in half the time of the typical training pipeline, providing value to the company and team members.\n- Developed and implemented Python software to automate routine tasks, increasing the productivity of a 2-man team threefold.\n- Represented the company at a networking event, resulting in new contractor connections and bids for upcoming projects in the greater metropolitan area.\n- Communicated effectively with lead contractors during onsite walkthroughs, leading to the successful awarding of a Rutgers Camden Lab renovation project.\n\nHeader 1: Experience\nHeader 2: Estimating Eng